# QLoRA aplicado a un LLM conocido como TinyLlama
En este cuadernillo se explica como se logra aplicar QLoRA a un Modelo preetrenado como lo es TinyLlama

Es fundamental para QLoRA ya que mantendremos el modelo base cuantizado (frozen) mientras solo se entrenan los adaptadores LoRA en precisión completa, logrando un balance óptimo entre eficiencia de memoria y calidad del fine-tuning.

## Importacion de librerias

En esta parte importamos todas las librerias necesarias para trabajar con QLoRA

In [2]:
import torch
import torch.nn as nn
import bitsandbytes as bnb # Para la cuantización
from torchvision import models, transforms, datasets
import matplotlib.pyplot as plt

from transformers import AutoTokenizer, AutoModelForCausalLM # Para cargar modelos de lenguaje
from peft import LoraConfig, get_peft_model # Para QLoRA

import torch
import bitsandbytes as bnb

d:\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Carga del modelo

En esta sección del código se está configurando y cargando un modelo de lenguaje grande (LLM) con cuantización de 4 bits para implementar QLoRA (Quantized Low-Rank Adaptation). Específicamente:

Se utiliza el modelo "PY007/TinyLlama-1.1B-Chat-v0.1", que es una versión compacta de 1.1 mil millones de parámetros optimizada para chat. Este modelo sirve como base para el fine-tuning posterior.

- **Cuantización a 4 bits:** El parámetro load_in_4bit=True activa la cuantización que reduce cada peso del modelo de 16 bits (float16) a solo 4 bits, disminuyendo dramáticamente el uso de memoria. Esto permite ejecutar modelos grandes en hardware con memoria limitada, como GPUs de consumo.

- **Configuración de precisión:** Se establece torch_dtype=torch.float16 para usar precisión half, mientras que bnb_4bit_compute_dtype=torch.float16 especifica que los cálculos internos también usen 16 bits. El parámetro device_map="auto" distribuye automáticamente las capas del modelo entre GPU y CPU según la memoria disponible.

- **Optimizaciones de cuantización:** bnb_4bit_use_double_quant=True habilita doble cuantización para mayor compresión, y bnb_4bit_quant_type="nf4" usa el formato NormalFloat4, que es especialmente efectivo para pesos que siguen distribuciones normales como los de los transformers.

In [3]:
# Nombre del modelo TinyLlama
model_name = "PY007/TinyLlama-1.1B-Chat-v0.1"

# Cargar el tokenizador para el modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Cargar el modelo con cuantización en 4 bits
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True, # Habilitar carga en 4 bits
    device_map="auto", # Asignar automáticamente a los dispositivos disponibles
    torch_dtype=torch.float16, # Usar float16 para eficiencia
    bnb_4bit_compute_dtype=torch.float16, # Tipo de dato para cálculos
    bnb_4bit_use_double_quant=True, # Usar doble cuantización para mejor precisión
    bnb_4bit_quant_type="nf4" # Tipo de cuantización (Normal Float 4)
)

d:\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Animetx\.cache\huggingface\hub\models--PY007--TinyLlama-1.1B-Chat-v0.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HT

### Configuracoin de los adaptadores LoRA

Se crea un objeto LoraConfig que define los parámetros clave para la adaptación de bajo rango:

- **r=8:** Este es el rango de las matrices de bajo rango A y B. Un valor de 8 significa que cada adaptador LoRA tendrá matrices de dimensión reducida, lo que mantiene el número de parámetros entrenables muy bajo comparado con el fine-tuning completo.

- **lora_alpha=32:** Factor de escalado que controla la influencia de los adaptadores LoRA. La relación alpha/r (32/8 = 4) determina qué tanto impacto tendrán las adaptaciones sobre el comportamiento original del modelo.

- **target_modules=["q_proj","v_proj"]:** Especifica exactamente qué capas del transformer serán adaptadas. En este caso, solo las proyecciones de query (q_proj) y value (v_proj) del mecanismo de atención, que son las más críticas para el comportamiento del modelo.

In [4]:
# Configuración LoRA
lora_config = LoraConfig(
    r=8,                 # rango bajo para adaptadores
    lora_alpha=32,       # escala de los adaptadores
    target_modules=["q_proj","v_proj"],  # capas donde aplicar LoRA, aqui en las proyecciones de query y value
    lora_dropout=0.1, # dropout para regularización
    bias="none",       # no adaptar sesgos, solo pesos
    task_type="CAUSAL_LM" # tipo de tarea
)

# Aplicar LoRA al modelo cuantizado
model = get_peft_model(model, lora_config)

## Carga del Dataset

Este conjunto de datos es una traducción al español de ``alpaca_data_cleaned.json``, que es una traducción al español del famoso dataset Alpaca de Stanford. Este dataset contiene instrucciones y respuestas en formato conversacional, ideal para entrenar modelos de chat en español. Sacado desde ``hugging-face`` https://huggingface.co/datasets/bertin-project/alpaca-spanish

In [6]:
from datasets import load_dataset

# Dataset de prueba
dataset = load_dataset("bertin-project/alpaca-spanish")

# Usar solo el conjunto de entrenamiento
train_dataset = dataset["train"]

# Preprocesamiento de los datos
def preprocess(examples):
    # Formatear prompt + respuesta para cada ejemplo en el batch
    prompts = [f"### Human: {inst}\n### Assistant: {out}" 
               for inst, out in zip(examples['instruction'], examples['output'])] # Crear prompts
    tokenized = tokenizer(prompts, truncation=True, padding="max_length", max_length=128) # Tokenizar
    tokenized["labels"] = tokenized["input_ids"].copy() # Usar input_ids como labels
    return tokenized

train_dataset = train_dataset.map(preprocess, batched=True) # Preprocesar el dataset
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"]) # Formatear para PyTorch

d:\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Animetx\.cache\huggingface\hub\datasets--bertin-project--alpaca-spanish. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular H

In [7]:
# Mostrar un ejemplo
print(train_dataset[0])

{'input_ids': tensor([    1,   835, 12968, 29901, 18613,  2182, 29948, 28711, 25348, 29973,
           13,  2277, 29937,  4007, 22137, 29901, 25348, 28711,  3976, 13321,
          553,  2251,   381,  1091,   265,  1682,   280,  1417, 29889, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 

## Entrenamiento del Modelo

En esta sección del se configura y ejecuta el entrenamiento del modelo QLoRA. Específicamente:

Configuración de argumentos de entrenamiento: Se crea un objeto TrainingArguments que define todos los hiperparámetros y configuraciones para el proceso de entrenamiento:

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./tinyllama-lora", # directorio de salida
    per_device_train_batch_size=2, # tamaño de batch por dispositivo
    gradient_accumulation_steps=8, # acumular gradientes
    learning_rate=2e-4, # tasa de aprendizaje
    fp16=True,               # usar precisión mixta
    logging_steps=10,       # pasos de registro
    save_steps=200,         # pasos para guardar el modelo
    save_total_limit=2,    # límite total de modelos guardados
    max_steps=1000,        # pasos máximos de entrenamiento
)
# Configurar el Trainer
trainer = Trainer(
    model=model, # el modelo a entrenar
    args=training_args, # argumentos de entrenamiento
    train_dataset=train_dataset # conjunto de datos de entrenamiento
)
# Iniciar el entrenamiento
trainer.train()

Step,Training Loss
10,5.764700
20,5.836300
30,5.124900
40,5.011300
50,4.614000
60,4.299900
70,4.446800
80,4.528800
90,4.303000
100,4.306700


TrainOutput(global_step=1000, training_loss=3.9183070449829103, metrics={'train_runtime': 1084.4133, 'train_samples_per_second': 14.755, 'train_steps_per_second': 0.922, 'total_flos': 1.2725954543616e+16, 'train_loss': 3.9183070449829103, 'epoch': 0.30803588618074007})

## Prueba del Modelo Entrenado con QLoRA


In [8]:
# Guardar adaptadores LoRA
model.save_pretrained("./tinyllama-lora")

In [8]:
# Cargar el modelo cuantizado y aplicar LoRA para inferencia
from peft import PeftModel

# Cargar el modelo cuantizado
model = AutoModelForCausalLM.from_pretrained(
    model_name, # nombre del modelo
    load_in_4bit=True, # cargar en 4 bits
    device_map="auto", # asignación automática de dispositivos
    torch_dtype=torch.float16, # usar float16
    bnb_4bit_compute_dtype=torch.float16, # tipo de dato para cálculos
    bnb_4bit_use_double_quant=True, # usar doble cuantización
    bnb_4bit_quant_type="nf4" # tipo de cuantización
)

# Cargar los adaptadores LoRA entrenados
model = PeftModel.from_pretrained(model, "./tinyllama-lora")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [13]:
from transformers import AutoTokenizer

# Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Crear un prompt de prueba
prompt = "### Human: ¿Capitan Lara usara uniforme?.\n### Assistant:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")


In [14]:
# Modo evaluación
model.eval()

# Generación de texto
with torch.no_grad():
    outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,  # importante para modelos pequeños
    eos_token_id=tokenizer.eos_token_id,
)

# Decodificar tokens
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Resultado:")
print(generated_text)


Resultado:
### Human: ¿Capitan Lara usara uniforme?.
### Assistant: No, Capitan Lara no usara uniforme.Si un capitan Lara usara uniforme, esto es una desafortunada y deshonrada actitud. Esto puede hacerle perder el apoyo del equipo, y puede afectar la confianza de los demás en la empresa.Es importante que los empleados sean respetuosos y respetan los derechos de los demás, y no deben utilizar conductas que afecten la estabilidad y la confianza de los demás.La seguridad de los empleados y los demás es el principal objetivo de la empresa, y es importante que los empleados sean respetuosos y respetan los derechos de los demás.Las estrategias de seguridad de la empresa incluyen la creación de un ambiente seguro y la vigilancia de los recursos.Los empleados
